# WNTR Multi-species Water Quality Tutorial
The following tutorial replicates the simple multi-species example of chlorine decay taken from the 
 [EPANET-MSX user manual](https://cfpub.epa.gov/si/si_public_record_report.cfm?dirEntryId=358205&Lab=CESER). The Net3 example network from EPANET is used, and two 
different decay coefficients are assigned - one for each source of water.
The river uses decay coefficient k1, the lake uses decay coefficient k2, and 
the two values are an order of magnitude different. A parameter sensitivity is also performed
to look at the effects of different decay coefficients for the river source.

## Imports
Import WNTR and additional Python packages that are needed for the tutorial.
- Pprint is used to "pretty-print" data structures, making them more readable
- JSON is used to work with JSON data
- Matplotlib is used to create graphics

In [ ]:
from pprint import pprint
import json
import matplotlib.pyplot as plt

import wntr
from wntr.msx.elements import InitialQuality
from wntr.library.msx import MsxLibrary
from wntr.msx import io as msxio

# Water network model
The following section creates a `WaterNetworkModel` object from an EPANET input (INP) file and defines the multi-species reaction model.

## Create the water network model 
It is advised to remove the EPANET quality parameter to avoid conflicts in the water quality analysis.  

In [ ]:
# Create a WaterNetworkModel from an EPANET input (INP) file
wn = wntr.network.WaterNetworkModel("networks/Net3.inp")
# Set the water quality parameter to none
wn.options.quality.parameter = "NONE"

## Define a new multi-species (MSX) reaction model

In [ ]:
# Define the MSX model from the EPANET-MSX user manual
wn.add_msx_model()
wn.msx.title = "Multisource Chlorine Decay"
wn.msx.references.append(
    """(2023) Shang, F., L. Rossman, and J. Uber. 
"EPANET-MSX 2.0 User Manual". EPA/600/R-22/199"""
)

# Set the MSX options for the simumulation
wn.msx.options.area_units = "FT2"
wn.msx.options.rate_units = "DAY"
wn.msx.options.timestep = 300

## Add the MSX reaction dynamics information
For this tutorial, the reaction dynamics come from the EPANET-MSX user manual. Two species are tracked: free chlorine (Cl2) and a tracer (T1). The tracer is used to select the appropriate decay coefficient. The river is source 1, and the lake is source 2.

The amount of free chlorine is based on the reaction rate equation:

$$
    \frac{d}{dt}\mathrm{Cl_2} = -(k_1 T_1 + k_2(1-T_1)) \mathrm{Cl_2}
$$

In [ ]:
# Define the MSX species, T1 and CL2, to be tracked in the simulation
T1 = wn.msx.add_species("T1", "bulk", units="MG", note="Source 1 Tracer")
Cl2 = wn.msx.add_species("CL2", "bulk", units="MG", note="Free Chlorine")
# Print the parameters for T1 and Cl2
print(repr(T1))
print(repr(Cl2))

In [ ]:
# Define the decay coefficient for each water source
k1 = wn.msx.add_constant("k1", 1.3, units="1/day")
k2 = wn.msx.add_constant("k2", 17.7, units="1/day")
# Print the parameters for k1 and k2
print(repr(k1))
print(repr(k2))

In [ ]:
# Define the reaction equations for T1 and Cl2
rxn_T1 = wn.msx.add_reaction("T1", "pipe", "rate", "0")
rxn_Cl2 = wn.msx.add_reaction("CL2", "pipe", "rate", "-(k1*T1 + k2*(1-T1))*CL2")
# Print the parameters for the Cl2 reaction
print(repr(rxn_Cl2))

## Set the initial quality values for the species
The initial quality values are set for the two species being simulated. The river's tracer, T1, is set to 1.0. The free chlorine species, Cl2, is set to 1.2 mg/L at both sources (river and lake).

In [ ]:
# Define the initial water quality value for T1 and Cl2
net_data = wn.msx.network_data
net_data.initial_quality["T1"] = InitialQuality(node_values={"River": 1.0})
net_data.initial_quality["CL2"] = InitialQuality(node_values={"River": 1.2, 
                                                              "Lake": 1.2})
# Print the values for Cl2 and T1
pprint(net_data.initial_quality)

# Hydraulic and water quality simulations
The following section simulates the hydraulics and and water quality using the MSX model. Simulation results are stored for later analysis.

## Simulate the hydraulics and water quality
With the MSX model attached to the `WaterNetworkModel`, the EpanetSimulator is called in the same manner as a non-MSX water quality simulation. The results are saved in keys with the species' name.

In [ ]:
# Simulate the hydraulics and water quality using EPANET
sim = wntr.sim.EpanetSimulator(wn)
res = sim.run_sim()
# Print the available result categories for the junctions and links
print("Node results:", ", ".join([k for k in res.node.keys()]))
print("Link results:", ", ".join([k for k in res.link.keys()]))

## View the results
The concentration of the multiple water quality species in the network can be printed and displayed over space and time. 

In [ ]:
# Plot the fraction of water coming from the river using the tracer species, T1, 
# for each junctions at hour 12, 24, and 36 of the simulation 
for plot_hr in [12, 24, 36]: 
    wntr.graphics.plot_network(
        wn,
        node_attribute=res.node["T1"].loc[3600 * plot_hr, :],
        title=f"{plot_hr} h",
        node_colorbar_label="River\nfraction",
)

# Plot the concentration of Cl2 and T1 over time for a specific junction
# Additional junction names are provided as examples
junction_name = "117"  # '191', '269', '117'
res.node["CL2"][junction_name].plot()
res.node["T1"][junction_name].plot()
plt.title("Node {}\nk1 = {:.1f}, k2 = {:.1f}".format(junction_name, k1.value, k2.value))
_ = plt.legend(["Cl2", "T1"])
_ = plt.xlabel('Time (s)')
_ = plt.ylabel('Concentration (mg/L)')

# Changing reaction parameters


In [ ]:
# Create a loop that stores several values for the river's decay coefficient, k1
# Iterate through each coefficient value and rerun the simulation
results = dict()
k1 = wn.msx.reaction_system.constants["k1"]
for i in range(7):
    # Create new decay coefficient, new_k1
    new_k1 = 1.3 + i * 2.6
    k1.value = new_k1
    res  = sim.run_sim()
    results[new_k1] = res.node["CL2"]

In [ ]:
# Plot the free chlorine, Cl2, concentration over time for the junction using
# all of the simulated k1 values
for k1, res in results.items():
    res.loc[0 : 3600 * 36, junction_name].plot()
plt.legend(["{:.1f}".format(k1) for k1 in results.keys()], title="k1 (1/day)")
plt.title("Chlorine residual at node {}".format(junction_name))
plt.xlabel("Time (s)")
plt.ylabel("Concentration (mg/L)")

# Save results to CSV files
Save chlorine residual results for each reaction rate value

In [ ]:
for k1, res in results.items():
    res.to_csv('chlorine_residual_'+str(k1)+'.csv')

# Saving the reaction dynamics model
The reaction dynamics model can be saved in multiple formats, which include the EPANET-MSX style format and a JSON file.
This model can also be saved in a library format, which strips the JSON file of any network-specific information so that it contains only the species, constants, and reaction dynamics, which can then be applied to any `WaterNetworkModel`.

In [ ]:
# Save the reaction dynamics model as a MSX file, Net3.msx
msxio.write_msxfile(wn.msx, "Net3.msx")
# Save the reaction dynamics model as a JSON file, Net3-msx.json
msxio.write_json(wn.msx, "Net3-msx.json")
# Save the reaction dynamics model as a library
msxio.write_json(wn.msx, "multisource-cl.json", as_library=True)

In [ ]:
# Open and print the content of the saved MSX file to confirm the information
with open("Net3.msx", "r") as fin:
    print(fin.read())

In [ ]:
# Open the JSON and the library files 
with_net: dict = None
without_net: dict = None

with open("Net3-msx.json", "r") as fin:
    with_net = json.load(fin)
with open("multisource-cl.json", "r") as fin:
    without_net = json.load(fin)

# Print the content of the JSON file
print("With network data:")
pprint(with_net["network_data"])

# Print the content of the library file 
# Compare against the JSON file data to confirm network data has been removed
print("As a library:")
pprint(without_net["network_data"])

# Using the WNTR MSX library
WNTR now includes a library functionality that allows a user to access certain objects by name.
The MSX integration includes adding a library of certain reaction models that are described in
the EPANET-MSX user manual. This section demonstrates how to use the model that was just saved
in the library.

In [ ]:
# Load the MSX library files from the current directory
# Includes MSX and JSON file formats
my_library = MsxLibrary(extra_paths=["."])  
# Print the names of the MSX library files from the current directory
my_library.model_name_list()

Note that more files are listed than what might have been expected because any file that has the extension .json or .msx is included. Thus, the first five models included in WNTR are listed along with those in the current directory ('.'), Net3, Net3-msx, multisource-cl, and "temp" files (since they were just created/saved in this tutorial). The reaction dynamic models are accessed by name. 

In [ ]:
# Compare the initial quality for the "Net3" and "multisource-cl" models
iq_Net3 = my_library.get_model("Net3").network_data.initial_quality
print(iq_Net3)
iq_multi = my_library.get_model("multisource-cl").network_data.initial_quality
print(iq_multi)

In [ ]:
# Load the "arsenic_chloramine" model 
arsenic = my_library.get_model("arsenic_chloramine")
# Print the variables (e.g., species, constants) and reaction 
# equations in this model
for key, value in arsenic.reaction_system.variables():
    print(repr(value))
for key, value in arsenic.reaction_system.reactions():
    print(repr(value))